In [ ]:
# https://coderzcolumn.com/tutorials/artificial-intelligence/how-to-use-glove-embeddings-with-pytorch

In [1]:
import torch

print("PyTorch Version : {}".format(torch.__version__))

PyTorch Version : 1.11.0+cu102


In [2]:
import torchtext

print("Torch Text Version : {}".format(torchtext.__version__))

Torch Text Version : 0.12.0


In [3]:
from torchtext.data import get_tokenizer

tokenizer = get_tokenizer("basic_english") ## We'll use tokenizer available from PyTorch

tokenizer("Hello, How are you?")

['hello', ',', 'how', 'are', 'you', '?']

In [4]:
from torchtext.vocab import GloVe
dim = 300
# global_vectors = GloVe(name='twitter.27B', dim=dim)
global_vectors = GloVe(name='840B', dim=dim)

.vector_cache/glove.840B.300d.zip: 2.18GB [07:13, 5.02MB/s]                                
100%|█████████▉| 2196016/2196017 [02:33<00:00, 14263.24it/s]


In [8]:
embeddings = global_vectors.get_vecs_by_tokens(tokenizer("Hello, How are you?"), lower_case_backup=True)

embeddings.shape

torch.Size([6, 100])

In [18]:
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

max_words = 25
embed_len = dim

def vectorize_batch(batch):
    Y, X = list(zip(*batch))
    X = [tokenizer(x) for x in X]
    X = [tokens+[""] * (max_words-len(tokens))  if len(tokens)<max_words else tokens[:max_words] for tokens in X]
    X_tensor = torch.zeros(len(batch), max_words, embed_len)
    for i, tokens in enumerate(X):
        X_tensor[i] = global_vectors.get_vecs_by_tokens(tokens)
    return X_tensor.reshape(len(batch), -1), torch.tensor(Y) - 1 ## Subtracted 1 from labels to bring in range [0,1,2,3] from [1,2,3,4]

target_classes = ["World", "Sports", "Business", "Sci/Tech"]

train_dataset, test_dataset  = torchtext.datasets.AG_NEWS()
train_dataset, test_dataset = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)

train_loader = DataLoader(train_dataset, batch_size=8, collate_fn=vectorize_batch)
test_loader  = DataLoader(test_dataset, batch_size=8, collate_fn=vectorize_batch)

/home/eyalshw/git/courses/venv/lib/python3.8/site-packages/torch/utils/data/datapipes/utils/common.py:24: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn(


In [25]:
next(iter(train_dataset))

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [27]:
for data in train_dataset:
    print(data)
    break

(3, "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")


In [ ]:
for X, Y in train_loader:

    print(X.shape, Y.shape)
    break

In [19]:
for X, Y in train_loader:

    print(X.shape, Y.shape)
    break

torch.Size([8, 2500]) torch.Size([8])


In [20]:
from torch import nn
from torch.nn import functional as F

class EmbeddingClassifier(nn.Module):
    def __init__(self):
        super(EmbeddingClassifier, self).__init__()
        self.seq = nn.Sequential(
            nn.Linear(max_words*embed_len, 256),
            nn.ReLU(),

            nn.Linear(256,128),
            nn.ReLU(),

            nn.Linear(128,64),
            nn.ReLU(),

            nn.Linear(64, len(target_classes)),
        )

    def forward(self, X_batch):
        return self.seq(X_batch)

In [21]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc

def CalcValLossAndAccuracy(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_shuffled.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f}".format(torch.tensor(losses).mean()))
        print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled.detach().numpy(), Y_preds.detach().numpy())))

def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=1):
    for i in range(1, epochs+1):
        losses = []
        for X, Y in tqdm(train_loader):
            print(X.shape)
            Y_preds = model(X)

            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if i%5==0:
            print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
            CalcValLossAndAccuracy(model, loss_fn, val_loader)

In [22]:
from torch.optim import Adam

epochs = 25
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
embed_classifier = EmbeddingClassifier()
optimizer = Adam(embed_classifier.parameters(), lr=learning_rate)

TrainModel(embed_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)

  0%|          | 22/15000 [00:00<01:10, 212.55it/s]

torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])


  0%|          | 67/15000 [00:00<01:08, 217.43it/s]

torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])


  1%|          | 111/15000 [00:00<01:08, 217.06it/s]

...

torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])


  5%|▍         | 740/15000 [00:03<01:11, 199.78it/s]

torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])
torch.Size([8, 2500])


  5%|▍         | 746/15000 [00:03<01:06, 215.31it/s]


KeyboardInterrupt: 